In [8]:
from neuralop.models import FNO, UNO
from neuralop.utils import count_model_params
import torch 

In [21]:
model = UNO(1,640, hidden_channels=24, projection_channels=8,uno_out_channels = [16,8,8,8,16], \
            uno_n_modes= [[64,64],[32,32],[32,32],[32,32],[64,64]], uno_scalings=  [[1.0,1.0],[0.5,0.5],[1,1],[2,2],[1,1]],\
            horizontal_skips_map = None, n_layers = 5, domain_padding = 1)
model = model.to("cuda")

n_params = count_model_params(model)
print(f'\nOur model has {n_params} parameters.')


Our model has 3606232 parameters.


In [22]:
trunk_nn = FNO(n_modes=(64,64),hidden_channels=12,in_channels=1,out_channels=640)
count_model_params(trunk_nn)

2605052

In [23]:
x = torch.randn(1, 1, 1, 300).to("cuda")

In [24]:
u_out = model(x)

In [25]:
u_out.shape

torch.Size([1, 640, 1, 300])

In [2]:

import operator
from functools import reduce
from functools import partial

# print the number of parameters
def count_params(model):
    c = 0
    for p in list(model.parameters()):
        c += reduce(operator.mul, 
                    list(p.size()+(2,) if p.is_complex() else p.size()))
    return c


In [8]:
"""
This code belongs to the paper:
-- Tripura, T., & Chakraborty, S. (2022). Wavelet Neural Operator for solving 
   parametric partialdifferential equations in computational mechanics problems.
   
-- This code is for 2-D Darcy equation in triangular domain with notch (time-independent problem).
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

from timeit import default_timer

from wavelet_convolution import WaveConv2d

torch.manual_seed(0)
np.random.seed(0)

# %%
""" The forward operation """
class WNO2d(nn.Module):
    def __init__(self, width, level, layers, size, wavelet, in_channel, grid_range, padding=0):
        super(WNO2d, self).__init__()

        """
        The WNO network. It contains l-layers of the Wavelet integral layer.
        1. Lift the input using v(x) = self.fc0 .
        2. l-layers of the integral operators v(j+1)(x,y) = g(K.v + W.v)(x,y).
            --> W is defined by self.w; K is defined by self.conv.
        3. Project the output of last layer using self.fc1 and self.fc2.
        
        Input : 3-channel tensor, Initial input and location (a(x,y), x,y)
              : shape: (batchsize * x=width * x=height * c=3)
        Output: Solution of a later timestep (u(x,y))
              : shape: (batchsize * x=width * x=height * c=1)
        
        Input parameters:
        -----------------
        width : scalar, lifting dimension of input
        level : scalar, number of wavelet decomposition
        layers: scalar, number of wavelet kernel integral blocks
        size  : list with 2 elements (for 2D), image size
        wavelet: string, wavelet filter
        in_channel: scalar, channels in input including grid
        grid_range: list with 2 elements (for 2D), right supports of 2D domain
        padding   : scalar, size of zero padding
        """

        self.level = level
        self.width = width
        self.layers = layers
        self.size = size
        self.wavelet = wavelet
        self.in_channel = in_channel
        self.grid_range = grid_range 
        self.padding = padding
        
        self.conv = nn.ModuleList()
        self.w = nn.ModuleList()
        
        self.fc0 = nn.Linear(self.in_channel, self.width) # input channel is 3: (a(x, y), x, y)
        for i in range( self.layers ):
            self.conv.append( WaveConv2d(self.width, self.width, self.level, self.size, self.wavelet) )
            self.w.append( nn.Conv2d(self.width, self.width, 1) )
        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)    
        x = self.fc0(x)                      # Shape: Batch * x * y * Channel
        x = x.permute(0, 3, 1, 2)            # Shape: Batch * Channel * x * y
        if self.padding != 0:
            x = F.pad(x, [0,self.padding, 0,self.padding]) 
        
        for index, (convl, wl) in enumerate( zip(self.conv, self.w) ):
            x = convl(x) + wl(x) 
            if index != self.layers - 1:     # Final layer has no activation    
                x = F.mish(x)                # Shape: Batch * Channel * x * y
                
        if self.padding != 0:
            x = x[..., :-self.padding, :-self.padding]     
        x = x.permute(0, 2, 3, 1)            # Shape: Batch * x * y * Channel
        x = F.gelu( self.fc1(x) )            # Shape: Batch * x * y * Channel
        x = self.fc2(x)                      # Shape: Batch * x * y * Channel
        return x
    
    def get_grid(self, shape, device):
        # The grid of the solution
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, self.grid_range[0], size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, self.grid_range[1], size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)


# %%
""" Model configurations """

PATH = 'data/Darcy_Triangular_FNO.mat'
ntrain = 1
ntest = 1

batch_size = 1
learning_rate = 0.001

epochs = 5
step_size = 50   # weight-decay step size
gamma = 0.5      # weight-decay rate

wavelet = 'db6'  # wavelet basis function
level = 2    # lavel of wavelet decomposition
width = 14   # uplifting dimension
layers = 6    # no of wavelet layers

sub = 2          # subsampling rate
h = int(((101 - 1)/sub) + 1) # total grid size divided by the subsampling rate
grid_range = [1, 1]          # The grid boundary in x and y direction
in_channel = 3  # (a(x, y), x, y) for this case

# %%
""" Read data """
x= 2
s = 300  #[batch,1,1,300]
x_train = torch.randn(1,x,s,1).cuda()
x_test = torch.randn(1,x,s,1).cuda()

y_train = torch.randn(1,x,s,1).cuda()
y_test = torch.randn(1,x,s,1).cuda()
# x_train = x_train.reshape(ntrain,h,h,1)
# x_test = x_test.reshape(ntest,h,h,1)
print("t")
# %%
device = "cuda"
""" The model definition """
model = WNO2d(width=width, level=level, layers=layers, size=[x,s], wavelet=wavelet,
              in_channel=3, grid_range=grid_range, padding=0).to(device)
print(count_params(model))

print("hi")


out = model(x_train)
print(out.shape)
        

t
